In [14]:
import json
#delivery rate
def getDeliveryRate(model, numPlan):
    delivered = 0
    with open (f'Output/{model}/plans/toolUsePlans.jsonl','r') as f:
         plans = [json.loads(line.strip()) for line in f]

    for record in plans:
        if record['plan'] != "":
            delivered += 1
    return delivered / numPlan

#need revision to be more precise
def getInvalidLoopAndWrongOrderRate(model):
    allError = 0
    invalidArgDeadLoop = 0
    wrongOrder = 0

    stoppedList = []
    with open (f'Output/{model}/plans/toolUseLogs.jsonl','r') as f:
        logs = [json.loads(line.strip()) for line in f]
    for log in logs:
        for step in reversed(log['log']):
            if step['state'] != 'Successful':
                #print(log['index'], step['step'],step['state'], step['action'])
                if step['state'] == 'same action 3 times repeated':
                    allError += 1
                    stoppedList.append([log['index'],step['step']])

    for stop in stoppedList:
        plan = stop[0] - 1
        index = stop[1] - 2

        if logs[plan]['log'][index]['state'] == 'Illegal args. Parameter Error':
            invalidArgDeadLoop += 1
        elif logs[plan]['log'][index]['state'] == 'Illegal args. Other Error':
            wrongOrder += 1
        else:
            continue


    #print(stoppedList)
    return allError, invalidArgDeadLoop, wrongOrder

def getRightFunctionCall(model):
    rightRestaurantSearch = 0
    rightAccommodationSearch = 0
    rightAttractionSearch = 0
    with open (f'Output/{model}/plans/toolUseLogs.jsonl','r') as f:
        logs = [json.loads(line.strip()) for line in f]
    for log in logs:
        for step in reversed(log['log']):
            if step['state'] == 'Successful':
                if 'Accommodation' in step['action']:
                     rightAccommodationSearch += 1
                     break
        
        for step in reversed(log['log']):
            if step['state'] == 'Successful':
                if 'Restaurant' in step['action']:
                     rightRestaurantSearch += 1
                     break
        
        for step in reversed(log['log']):
            if step['state'] == 'Successful':
                if 'Attraction' in step['action']:
                     rightAttractionSearch += 1
                     break
    
    return rightAccommodationSearch, rightRestaurantSearch, rightAttractionSearch

if __name__ == "__main__":
    model = 'llama318b'
    deliverayRate = getDeliveryRate('llama318b',300)
    print(f"the delivery rate for {model}, is:" ,deliverayRate)

    allError,invalidArgDeadLoop,wrongOrder = getInvalidLoopAndWrongOrderRate(model)
    print(f"{model} have {allError} error stops. In them, {invalidArgDeadLoop} are dead loop due to invalid argument function call, {wrongOrder} are call the function in wrong order.")

    acco, res, right = getRightFunctionCall(model)
    print(f"rightAccommodationSearch: {acco}, rightRestaurantSearch: {res}, rightAttractionSearch: {right}")

the delivery rate for llama318b, is: 0.8466666666666667
llama318b have 46 error stops. In them, 22 are dead loop due to invalid argument function call, 23 are call the function in wrong order.
rightAccommodationSearch: 272, rightRestaurantSearch: 276, rightAttractionSearch: 294
